In [1]:
from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier
from tabpfn.scripts.transformer_prediction_interface import transformer_predict

model = TabPFNClassifier()

# import train test split
from sklearn.model_selection import train_test_split

import torch

Using style prior: True
{'recompute_attn': True}
<module 'wandb' from '/home/parietal/lgrinszt/.local/miniconda3/envs/tab_pfn2/lib/python3.9/site-packages/wandb/__init__.py'>
Using cpu device
Batch size: 1
Using distributed training: False
Using a Transformer with 25.82 M parameters


In [4]:
# Feed an input to the model and check that the differentation works
# Create a random input
input = torch.rand(200, 1, 100)
# Create a random binary target
target = torch.randint(0, 2, (200, 1)).float()

# Differentiate the input
input.requires_grad = True

# Feed the input to the model
#output = model.fit(input, target) #Doesn't work
module = model.model[2]
module.efficient_eval_masking = True
module.full_attention = False
single_eval_pos = 10
output = module((input, target), single_eval_pos=single_eval_pos) #Works

torch.Size([200, 1, 512])
torch.Size([200, 1, 512])
torch.Size([10, 1, 512])
torch.Size([10, 1, 512])
torch.Size([200, 1, 512])
torch.Size([200, 1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([200, 1, 512])
torch.Size([200, 1, 10])


In [76]:
# Create fake datasets
# Create a first dataset with columns A, B
dataset1 = torch.rand(1000, 1, 100)
# Create a second dataset with columns C, D
dataset2 = torch.rand(1000, 1, 100)

# Make column C equal to column A * 2 + noise
dataset2[:, 0] = dataset1[:, :, 0] * 2 #+ torch.rand(1000) * 0.1

# Create y as D + noise
y = dataset2[:, :, 1] #+ torch.rand(1000) * 0.1

dataset2.requires_grad = True
dataset1.requires_grad = True

# Make it a classification problem
y = (y > 0.5).float()

In [12]:
# Try TabPFN on dataset1
model = TabPFNClassifier()
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(dataset2, y, test_size=0.2, random_state=42)
# Fit the model
model.fit(X_train, y_train)
# Predict on the test set
y_pred = model.predict(X_test)
print(y_pred)
# Check the accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))


Using style prior: True
{'recompute_attn': True}
<module 'wandb' from '/home/parietal/lgrinszt/.local/miniconda3/envs/tab_pfn2/lib/python3.9/site-packages/wandb/__init__.py'>
Using cpu device
Batch size: 1
Using distributed training: False
Using a Transformer with 25.82 M parameters
interface
torch.Size([1000, 1, 100])
torch.Size([1000, 1])
800
torch.Size([1000, 3, 512])
torch.Size([800, 3, 512])
torch.Size([800, 3, 512])
torch.Size([800, 3, 512])
torch.Size([1000, 3, 512])
torch.Size([1000, 3, 512])
torch.Size([3, 512])
torch.Size([3, 512])


/home/parietal/lgrinszt/.local/miniconda3/envs/tab_pfn2/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


torch.Size([1000, 3, 512])
torch.Size([1000, 3, 10])
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
0.71


In [12]:
from tabpfn.transformer_2 import TransformerModel2
from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier

model = TabPFNClassifier()
module = model.model[2]
config = model.c

model_torch = TransformerModel2(nhead=config["nhead"], nlayers=config["nlayers"],
                                encoder=module.encoder, y_encoder=module.y_encoder,
                                n_out=module.n_out, ninp=module.ninp, nhid=module.nhid,
                                efficient_eval_masking=module.efficient_eval_masking,
                                full_attention=module.full_attention)
# Transfer the weights
weights_to_transfer = model.model[2].state_dict()
weights_to_transfer.pop("criterion.weight")
model_torch.load_state_dict(weights_to_transfer)


Using style prior: True
{'recompute_attn': True}
<module 'wandb' from '/home/parietal/lgrinszt/.local/miniconda3/envs/tab_pfn2/lib/python3.9/site-packages/wandb/__init__.py'>
Using cpu device
Batch size: 1
Using distributed training: False
Using a Transformer with 25.82 M parameters


<All keys matched successfully>

In [62]:
dataset1

tensor([[[0.5668, 0.3454, 0.7760,  ..., 0.5050, 0.1498, 0.7433]],

        [[0.4660, 0.5265, 0.3188,  ..., 0.1435, 0.0943, 0.2766]],

        [[0.9613, 0.8483, 0.9688,  ..., 0.5763, 0.0273, 0.4668]],

        ...,

        [[0.3341, 0.6840, 0.5179,  ..., 0.3008, 0.4581, 0.4205]],

        [[0.3116, 0.3167, 0.6324,  ..., 0.7637, 0.0793, 0.2207]],

        [[0.3328, 0.2110, 0.3774,  ..., 0.6141, 0.2846, 0.4213]]],
       requires_grad=True)

In [68]:
#y_pred = transformer_predict(model.model[2], dataset1, y, single_eval_pos=600, eval_position=600,
#                                          preprocess_transform="none")
#print(model.model[2]((dataset2, y)))
print(model.model[2]((dataset2, y), single_eval_pos=200))
y_pred = torch.argmax(model.model[2]((dataset2, y), single_eval_pos=200), dim=-1).reshape(-1)
print(y_pred)
print(y_pred.shape)
# Check the accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(y[-len(y_pred):], y_pred))

torch.Size([1000, 1, 512])
torch.Size([1000, 1, 512])
torch.Size([200, 1, 512])
torch.Size([200, 1, 512])
torch.Size([1000, 1, 512])
torch.Size([1000, 1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1000, 1, 512])
torch.Size([1000, 1, 10])
tensor([[[  0.3168,  11.3697,  -8.4459,  ..., -14.3448, -10.8779,  -9.1673]],

        [[ 10.9316,   0.3576,  -5.8419,  ..., -14.1164, -13.4658, -12.1414]],

        [[  2.8302,  11.0658,  -8.6245,  ..., -13.8329, -12.5138, -11.6858]],

        ...,

        [[  2.3440,  11.5596,  -8.9687,  ..., -14.3094, -12.2946, -11.2831]],

        [[  2.7327,  11.2391,  -8.7390,  ..., -13.9931, -12.4805, -11.5873]],

        [[  2.3710,  11.5477,  -8.9633,  ..., -14.2959, -12.3061, -11.2988]]],
       grad_fn=<SliceBackward>)
torch.Size([1000, 1, 512])
torch.Size([1000, 1, 512])
torch.Size([200, 1, 512])
torch.Size([200, 1, 512])
torch.Size([1000, 1, 512])
torch.Size([1000, 1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1000, 1, 51

In [77]:
# import multihead attention
from torch.nn import MultiheadAttention
attention = MultiheadAttention(embed_dim=100, num_heads=1, dropout=0.0)
W_q = torch.rand(100, 100)
W_k = torch.rand(100, 100)
W_v = torch.rand(100, 100)
W_q.requires_grad = True
W_k.requires_grad = True
W_v.requires_grad = True

Q = torch.matmul(dataset1, W_q)
K = torch.matmul(dataset2, W_k)
V = torch.matmul(dataset2, W_v)

# Transform dataset1 by applying the attention to dataset2
dataset1 = attention(Q, K, V)[0]


In [81]:
dataset2.shape

torch.Size([1000, 1, 100])

In [82]:

y_pred = torch.argmax(model.model[2]((dataset1, y), single_eval_pos=200), dim=-1).reshape(-1)
print(y_pred)
print(y_pred.shape)
# Check the accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(y[-len(y_pred):], y_pred))

torch.Size([1000, 1, 512])
torch.Size([1000, 1, 512])
torch.Size([200, 1, 512])
torch.Size([200, 1, 512])
torch.Size([1000, 1, 512])
torch.Size([1000, 1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1000, 1, 512])
torch.Size([1000, 1, 10])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1,